In [1]:
# Falls noch nicht installiert:
# !pip install selenium beautifulsoup4 pandas

import re
import time
import pandas as pd
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup

# Passe den Pfad zu deinem ChromeDriver an
CHROMEDRIVER_PATH = "../drivers/chromedriver.exe"  # Beispiel: Ordner "drivers" im Projektverzeichnis


In [2]:
# Zelle 2: Funktion zur Abfrage der Seite und Extraktion der Rohdaten
def get_table_and_match_results(season_id, spieltag):
    """
    Ruft die Seite der Spieltagstabelle ab (z.B. Saison 24/25) und extrahiert:
      - Den aggregierten Tabellenbereich (die erste Tabelle mit class "items")
      - Die Rohzeilen der detaillierten Tagesresultate (im Bereich "responsive-table")
    Vorab wird geprüft, ob der Spieltag (über ein Datum in einem Link) bereits stattgefunden hat.
    """
    url = f"https://www.transfermarkt.ch/super-league/spieltagtabelle/wettbewerb/C1?saison_id={season_id}&spieltag={spieltag}"
    
    service = Service(CHROMEDRIVER_PATH)
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("window-size=1920,1080")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36")
    
    driver = webdriver.Chrome(service=service, options=options)
    driver.get(url)
    time.sleep(5)  # Warte, bis alle Inhalte geladen sind
    soup = BeautifulSoup(driver.page_source, "html.parser")
    driver.quit()
    
    # Prüfe Datum: suche in Links nach einem Datum im Format dd.mm.yyyy
    date_pattern = re.compile(r'\d{2}\.\d{2}\.\d{4}')
    match_date = None
    for a in soup.find_all("a", href=True):
        if "datum" in a['href']:
            candidate = a.get_text(strip=True)
            if date_pattern.match(candidate):
                try:
                    match_date = datetime.strptime(candidate, "%d.%m.%Y")
                    break
                except Exception:
                    continue
    today = datetime.today()
    if match_date and match_date > today:
        print(f"Spieltag {spieltag} für Saison {season_id} liegt in der Zukunft ({match_date.strftime('%d.%m.%Y')}). Überspringe diesen Tag.")
        return None, None
    
    # Aggregierte Tabelle extrahieren: Wir nutzen eine Hilfsfunktion, die aus jeder Zeile (TR) genau 10 Zellen (TD) verarbeitet
    league_table = []
    table = soup.find("table", class_="items")
    if table:
        tbody = table.find("tbody")
        if tbody:
            rows = tbody.find_all("tr")
            for row in rows:
                parsed = parse_league_row(row)
                if parsed:
                    # Füge Season und Spieltag vorne ein
                    parsed.insert(0, spieltag)
                    parsed.insert(0, season_id)
                    league_table.append(parsed)
    else:
        print("Keine aggregierte Tabelle mit class 'items' gefunden!")
    
    # Detaillierte Tagesresultate extrahieren:
    match_rows = []
    responsive_div = soup.find("div", class_="responsive-table")
    if responsive_div:
        table2 = responsive_div.find("table")
        if table2:
            tbody2 = table2.find("tbody")
            if tbody2:
                # Sammle alle TRs – diese enthalten sowohl Header-Zeilen (mit Klasse "bg_blau_20") als auch Spielzeilen
                match_rows = tbody2.find_all("tr")
    else:
        print("Keine detaillierten Tagesresultate gefunden!")
    
    return league_table, match_rows

def parse_league_row(row):
    """
    Extrahiert aus einer Zeile der aggregierten Tabelle:
      - Rank (aus der ersten Zelle)
      - Team (aus der dritten Zelle – Linktext)
      - Danach: Spiele, G, U, V, Tore, +/-, Pkt.
    Erwartet mindestens 10 TD-Elemente.
    """
    cells = row.find_all("td")
    if len(cells) < 10:
        return None
    rank = cells[0].get_text(strip=True)
    team = cells[2].get_text(strip=True)
    spiele = cells[3].get_text(strip=True)
    g = cells[4].get_text(strip=True)
    u = cells[5].get_text(strip=True)
    v = cells[6].get_text(strip=True)
    tore = cells[7].get_text(strip=True)
    goal_diff = cells[8].get_text(strip=True)
    points = cells[9].get_text(strip=True)
    return [rank, team, spiele, g, u, v, tore, goal_diff, points]

# Testaufruf (zum Beispiel für Saison 24/25, Spieltag 25)
league_data, raw_match_data = get_table_and_match_results(2024, 25)
print("Beispiel aggregierte Tabelle (roh):")
print(pd.DataFrame(league_data).head())


Beispiel aggregierte Tabelle (roh):
     0   1  2               3   4   5  6  7      8   9   10
0  2024  25  1        FC Basel  25  12  6  7  53:28  25  42
1  2024  25  2       FC Lugano  25  12  6  7  42:34   8  42
2  2024  25  3       FC Luzern  25  12  6  7  44:38   6  42
3  2024  25  4     Servette FC  25  11  9  5  41:35   6  42
4  2024  25  5  Lausanne-Sport  25  10  7  8  43:33  10  37


In [3]:
# Zelle 3: Funktion zum Parsen der detaillierten Tagesresultate

def parse_detailed_matches(match_rows):
    """
    Iteriert über die TR-Elemente der detaillierten Tabelle.
    Header-Zeilen (mit Klasse "bg_blau_20") enthalten Datum und Uhrzeit und setzen den Kontext.
    Reguläre Spielzeilen werden verarbeitet, wobei angenommen wird:
      - Index 3: Heimmannschaft (kurz) mit Rang (z.B. "(5.)LS")
      - Index 6: Ergebnis (im Format "x:y")
      - Index 8: Gastmannschaft (kurz) mit Rang (z.B. "(11.)GCZ")
    Die aktuellen Datum und Uhrzeit werden aus der zuletzt gefundenen Header-Zeile übernommen.
    """
    matches = []
    current_date = ""
    current_time = ""
    # Regex, um den Rang (in Klammern) zu extrahieren
    rank_pattern = re.compile(r'\(\s*(\d+)\s*\.\s*\)')
    
    for row in match_rows:
        classes = row.get("class", [])
        # Header-Zeile erkennen (z. B. Klasse "bg_blau_20")
        if "bg_blau_20" in classes:
            text = row.get_text(" ", strip=True)
            date_match = re.search(r'(\d{2}\.\d{2}\.\d{4})', text)
            time_match = re.search(r'(\d{1,2}:\d{2})', text)
            if date_match:
                current_date = date_match.group(1)
            if time_match:
                current_time = time_match.group(1)
            continue  # Header überspringen
        # Normale Spielzeile
        cells = row.find_all("td")
        # Wir erwarten mindestens 11 Zellen (siehe Struktur oben)
        if len(cells) < 11:
            continue
        
        # Ergebnis aus Zelle 6
        result_text = cells[6].get_text(strip=True)
        if not re.match(r'\d+:\d+', result_text):
            continue
        try:
            home_goals, away_goals = map(int, result_text.split(":"))
        except Exception:
            home_goals, away_goals = None, None
        
        # Für die Heimmannschaft (aus Zelle 3)
        home_info = cells[3].get_text(strip=True)
        home_rank_match = rank_pattern.search(home_info)
        home_rank = int(home_rank_match.group(1)) if home_rank_match else None
        home_team = rank_pattern.sub("", home_info).strip()

        # Für die Gastmannschaft (aus Zelle 8)
        away_info = cells[8].get_text(strip=True)
        away_rank_match = rank_pattern.search(away_info)
        away_rank = int(away_rank_match.group(1)) if away_rank_match else None
        away_team = rank_pattern.sub("", away_info).strip()
        
        match_dict = {
            "date": current_date,
            "time": current_time,
            "home_rank": home_rank,
            "home_team": home_team,
            "home_goals": home_goals,
            "away_goals": away_goals,
            "away_rank": away_rank,
            "away_team": away_team
        }
        matches.append(match_dict)
    return matches

# Test: Parsen der detaillierten Zeilen (sofern raw_match_data nicht leer ist)
if raw_match_data and len(raw_match_data) > 0:
    parsed_matches = parse_detailed_matches(raw_match_data)
    if parsed_matches:
        print("Beispiel geparste detaillierte Zeile:")
        print(parsed_matches[0])
    else:
        print("Keine detaillierten Spiele geparst.")


Beispiel geparste detaillierte Zeile:
{'date': '22.02.2025', 'time': '18:00', 'home_rank': 5, 'home_team': 'LS', 'home_goals': 2, 'away_goals': 2, 'away_rank': 11, 'away_team': 'GCZ'}


In [4]:
# Zelle 4: Iteration über alle Spieltage und Erstellung der finalen DataFrames
def get_all_data(seasons, start_day, end_day):
    league_all = []
    matches_all = []
    for season in seasons:
        for spieltag in range(start_day, end_day + 1):
            print(f"Verarbeite Spieltag {spieltag} für Saison {season}...")
            league_data, raw_match_rows = get_table_and_match_results(season, spieltag)
            if league_data is None:
                print(f"Abbruch bei Spieltag {spieltag} für Saison {season} (zukünftiger Spieltag).")
                break
            league_all.extend(league_data)
            detailed_matches = parse_detailed_matches(raw_match_rows)
            for match in detailed_matches:
                match["Season"] = season
                match["Spieltag"] = spieltag
                matches_all.append(match)
    return league_all, matches_all

# Beispiel: Alle Spieltage der Saisons 24/25 und 23/24 (Spieltag 1 bis 38)
league_all_data, matches_all_data = get_all_data([2024, 2023], 1, 38)

# Erstelle DataFrames
# Aggregierte Tabelle: Wir erwarten 9 Spalten aus parse_league_row plus Season und Spieltag → 11 Spalten
league_columns = ["Season", "Spieltag", "Rank", "Team", "Spiele", "G", "U", "V", "Tore", "Goal_Diff", "Points"]
df_league_table = pd.DataFrame(league_all_data, columns=league_columns)

# Detaillierte Tagesresultate: Wir erwarten 8 Felder aus parse_detailed_matches plus Season und Spieltag → 10 Spalten
match_columns = ["Season", "Spieltag", "date", "time", "home_rank", "home_team", "home_goals", "away_goals", "away_rank", "away_team"]
df_matches = pd.DataFrame(matches_all_data, columns=match_columns)

Verarbeite Spieltag 1 für Saison 2024...
Verarbeite Spieltag 2 für Saison 2024...
Verarbeite Spieltag 3 für Saison 2024...
Verarbeite Spieltag 4 für Saison 2024...
Verarbeite Spieltag 5 für Saison 2024...
Verarbeite Spieltag 6 für Saison 2024...
Verarbeite Spieltag 7 für Saison 2024...
Verarbeite Spieltag 8 für Saison 2024...
Verarbeite Spieltag 9 für Saison 2024...
Verarbeite Spieltag 10 für Saison 2024...
Verarbeite Spieltag 11 für Saison 2024...
Verarbeite Spieltag 12 für Saison 2024...
Verarbeite Spieltag 13 für Saison 2024...
Verarbeite Spieltag 14 für Saison 2024...
Verarbeite Spieltag 15 für Saison 2024...
Verarbeite Spieltag 16 für Saison 2024...
Verarbeite Spieltag 17 für Saison 2024...
Verarbeite Spieltag 18 für Saison 2024...
Verarbeite Spieltag 19 für Saison 2024...
Verarbeite Spieltag 20 für Saison 2024...
Verarbeite Spieltag 21 für Saison 2024...
Verarbeite Spieltag 22 für Saison 2024...
Verarbeite Spieltag 23 für Saison 2024...
Verarbeite Spieltag 24 für Saison 2024...
V

In [5]:
# Export code from the scraper notebook
df_matches.to_csv("../data/df_matches_raw.csv", index=False)
df_league_table.to_csv("../data/df_league_table_raw.csv", index=False)